# Music Recommendation Project

Short description here... this will be Algorithm Development: Import features, baseline algorithm, complex algorithm, hyper-parameter optimization

Author: Ben Walsh \
February 7, 2021

## Contents

1. [Feature Import](#feature-data-import)
2. Data Exploration
3. Data Cleaning
4. Feature Engineering
5. Feature Selection
6. [Saving Data](#6.-Save-Data)

## <a class="anchor" id="feature-data-import"></a>1. Feature Data Import

### Import libraries

In [2]:
import pandas as pd
import os
import xgboost as xgb

First import all cleaned feature data: X and y target data for training and testing.

In [18]:
X_train_file = './data-input-clean/X_train.csv'
y_train_file = './data-input-clean/y_train.csv'

### Import training data

In [19]:
if os.path.exists(X_train_file):
    X_train = pd.read_csv(X_train_file)
else:
    print('Training data file {} not found!'.format(X_train_file))

if os.path.exists(y_train_file):
    y_train = pd.read_csv(y_train_file)
else:
    print('Training data file {} not found!'.format(y_train_file))

## 2. Baseline Model

## 3. XGBoost Model

In [25]:
xgb_model = xgb.XGBRegressor(objective ='reg:linear', #reg:squarederror #?
                             colsample_bytree = 0.3, 
                             learning_rate = 0.1,
                             max_depth = 5, 
                             alpha = 10, 
                             n_estimators = 10)

In [26]:
xgb_model.fit(X_train, y_train)

[21:32:26] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:32:26] WARNING: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\learner.cc:686: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.


XGBRegressor(alpha=10, colsample_bytree=0.3, max_depth=5, n_estimators=10)

## 4. Evaluate Model

For simplicity for now, compare against training data

In [48]:
y_predict = xgb_model.predict(X_train)

In [49]:
# Round outputs to compare
y_predict = y_predict.round().reshape(len(y_predict),1)

In [55]:
print('Accuracy on training set = {:.2f}%'.format(100*(y_predict == y_train.values).sum() / len(y_predict)))

Accuracy on training set = 61.66%
